# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)

In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for AAPL to identify trends and predict market movements. I should delegate tasks accordingly and ask the right questions to get the best insights for significant market opportunities or threats.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements for AAPL. Ensure to provide insights and alerts about significant market opportunities or threats."}

> Entering new CrewAgentExecutor chain...
I need to gather real-time market da

Final Answer: 
Based on the current market data for AAPL, I recommend considering the following trading strategies:

1. Trend Following: Given the historical price movements and the current market data for AAPL, implementing a trend-following strategy could be beneficial. This strategy involves identifying and following the existing trends in the stock price, either upward or downward, to capitalize on potential profit opportunities.

2. Momentum Trading: With AAPL trading in the middle of its 52-week range and below its 200-day simple moving average, momentum trading could be a viable strategy. This approach involves buying stocks that are showing upward momentum and selling those that are showing downward momentum.

3. Volatility Breakout: Considering the intraday high and low prices of AAPL, a volatility breakout strategy could be effective. This strategy involves entering a trade when the price breaks out of its recent trading range, expecting a significant price movement in the di

Thought: I have gathered information on various day trading strategies for AAPL from the internet search results. I should evaluate these strategies based on the user-defined risk tolerance and preferences for day trading.

Final Answer:
After conducting research on day trading strategies for AAPL, I have found a variety of resources that provide insights into different approaches. These include leveraging trend movements, using technical indicators, and focusing on short-term price fluctuations. It is crucial to consider the user-defined risk tolerance and preferences for day trading when selecting the most suitable strategy. By incorporating these factors, we can develop and refine trading strategies for AAPL that align with the user's goals and objectives.

> Finished chain.
 

After conducting research on day trading strategies for AAPL, I have found a variety of resources that provide insights into different approaches. These include leveraging trend movements, using technical ind

Final Answer: 
Based on the market data analysis for AAPL from Yahoo Finance, the stock price for Apple Inc. (AAPL) is currently at $212.93 with a decrease of 1.07%. The market cap is at 3.199T, and the key events indicate a slight decrease in the stock performance over the past month and year. The company overview highlights Apple's core products and services, including iPhone, Mac, iPad, wearables, and services like Apple Music and Apple Pay. The profitability metrics show a healthy profit margin of 24.30% and strong financial performance. Overall, the analysis suggests a stable position for AAPL in the market.

> Finished chain.
 

Based on the market data analysis for AAPL from Yahoo Finance, the stock price for Apple Inc. (AAPL) is currently at $212.93 with a decrease of 1.07%. The market cap is at 3.199T, and the key events indicate a slight decrease in the stock performance over the past month and year. The company overview highlights Apple's core products and services, includin

Thought:
I have gathered valuable insights from the Data Analyst regarding the market data for AAPL and have received various trading strategies recommendations from the Trading Strategy Developer. Now, I need to compile detailed execution plans based on these inputs.

Final Answer:
Execution Plan for AAPL Trading:

1. Trend Following Strategy:
- Utilize the insights provided by the Data Analyst to identify and follow the trends in AAPL's stock price.
- Implement a position when the price is trending upwards and exit when the trend reverses.
- Set clear entry and exit points based on trend analysis to maximize profits.

2. Momentum Trading Strategy:
- Monitor the momentum of AAPL's stock price based on market conditions and news.
- Look for opportunities to enter positions when momentum is strong in a particular direction.
- Implement stop-loss orders to manage risks and protect profits in case of sudden reversals.

3. Volatility Breakout Strategy:
- Analyze the historical volatility o

Final Answer: Trend trading is a trading style that attempts to capture gains through the analysis of an asset's momentum in a particular direction. When the price is moving in one overall direction, such as up or down, that is called a trend. Trend traders enter into a long position when a security is trending upward, characterized by higher swing lows and higher swing highs. They may opt to enter a short position when an asset is trending lower, characterized by lower swing lows and lower swing highs.

Trend trading strategies assume that a security will continue to move in the same direction as it is currently trending. Such strategies often contain a take-profit or stop-loss provision to lock in profits or avoid big losses if a trend reversal occurs. Traders use both price action and other technical tools to determine the trend direction and when it may be shifting.

There are many different trend trading strategies, each using a variety of indicators and price action methods. Movi

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL Trading Strategies:

1. Trend Following Strategy:
- Potential Risks:
  - False breakouts leading to losses if the trend reverses quickly.
  - Unexpected market news causing sharp price movements against the trend.
- Mitigation Strategies:
  - Utilize trailing stop-loss orders to lock in profits and protect against abrupt reversals.
  - Regularly review trend indicators and adjust entry/exit points based on changing market conditions.

2. Momentum Trading Strategy:
- Potential Risks:
  - Sudden market volatility leading to stop-loss orders being triggered prematurely.
  - Limited liquidity during market open/close causing slippage in execution.
- Mitigation Strategies:
  - Implement dynamic stop-loss orders based on volatility levels to avoid premature exits.
  - Monitor market liquidity and adjust trade sizes accordingly to minimize slippage risks.

3. Volatility Breakout Strategy:
- Potential Risks:
  - False breakouts due to temporary price spikes triggering entry/exit signals.
  - High volatility leading to increased risk of price gaps during execution.
- Mitigation Strategies:
  - Confirm breakout signals with multiple volatility indicators to reduce false signals.
  - Use limit orders to control execution price and minimize slippage risks during high volatility periods.

4. Pair Trading Strategy:
- Potential Risks:
  - Correlation breakdown between AAPL and the paired asset causing unexpected losses.
  - Market events impacting one asset more than the other leading to unbalanced positions.
- Mitigation Strategies:
  - Regularly monitor correlation coefficients and adjust positions based on changing relationships.
  - Implement stop-loss orders on both long and short positions to limit potential losses from adverse events.

5. Risk Tolerance Alignment:
- Potential Risks:
  - User's risk tolerance not accurately reflected in position sizes and stop-loss levels.
  - Failure to adjust trading plan based on changing market conditions and risk assessments.
- Mitigation Strategies:
  - Conduct regular risk assessments to ensure alignment with user's risk preferences.
  - Implement a systematic review process to adapt trading strategies to evolving market conditions and risk profiles.

Overall, the proposed trading strategies for AAPL present various risks that require careful evaluation and proactive risk management. By implementing the suggested mitigation strategies, the trading team can effectively manage risks and maximize profits while trading AAPL. Continuously monitoring market conditions and adjusting strategies accordingly will be crucial for success in trading AAPL.